<a href="https://colab.research.google.com/github/realBarnaby/news/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Szövegfeldolgozás:
megadhatóak weboldalak,
a program egy nlp segítségével leszedi ezeket,
majd elküldi egy adott e-mailre

továbbá készíthető hozzá egy webapp
és egy adatbázis

bónusz? személyre szabási lehetőségek esetleg elkészíthető több nlpre, weboldalra, rovatra

# Modulok

In [ ]:
# importáljuk a newspaper3k modult, amely segít gyűjteni és feldolgozni a híreket
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=31d560e3e51c0b36cf06e837c1e199cc4593f9c2551bbe6e0e6ab67c3f4d73cd
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3340 sha256=753f44144c3a55d92bf7a9c69712863db2434b3a0c0a9609e7eae18907bfa801
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc

In [ ]:
# importáljuk a datetime modult, amely segít kezelni a dátumokat
import datetime

from newspaper import Article
import requests
from bs4 import BeautifulSoup
import pandas as pd

# importáljuk read timed out hibák elkerülése érdekében
from newspaper import Config

# Origo

In [ ]:
# dátum beállítása
# ---------------------------------------------------


# létrehozunk egy üres listát az url-ek tárolására
urls = []

# létrehozunk egy datetime objektumot 2024. április 1-jével
date = datetime.date(2024, 4, 1)  # megadva a felhasználó áltlal a webappban

# létrehozunk egy másik datetime objektumot 2024. aprilis 2-ával
end_date = datetime.date(2024, 4, 2)  # megadva a felhasználó áltlal a webappban

# végigmegyünk a dátumokon egy napos lépésekkel, amíg el nem érjük a végdátumot
while date < end_date:
    # kinyerjük a dátum évét, hónapját és napját
    year = date.year
    month = date.month
    day = date.day
    # összerakjuk az url-t a dátummal
    url = f"https://www.origo.hu/hirarchivum/{year}/{month}/{day}"
    # hozzáadjuk az url-t a listához
    urls.append(url)

    # növeljük a dátumot egy nappal
    date += datetime.timedelta(days=1)

# kiírjuk az url-ek listáját
print(urls)

['https://www.origo.hu/hirarchivum/2024/4/1']


In [ ]:
# hírek lekérése origon, a megszabott rovatok alapján
# ---------------------------------------------------


# egy lista a magyar politikai weboldalakról, amelyekről híreket szeretnél kapni
# megadva a felhasználó áltlal a webappban

# egy üres lista a hírek tárolására
news = []                         # belföldi (politikai) hírek
neutral_news = []                 # egyéb (nem annyira politikai) hírek

# egy lista a választható rovatok tárolásásra
preference = ['/itthon/', '/auto/', '/tudomany/', '/sport/']
#print("preference: ", preference)
input = preference[0]
#print("input: ", input)

# végigmegyünk a weboldalak listáján
for website in urls:
    # lekérjük a weboldal tartalmát
    response = requests.get(website)
    # létrehozunk egy BeautifulSoup objektumot a HTML elemzéséhez
    soup = BeautifulSoup(response.text, "html.parser")
    # megkeressük az összes hírcímet tartalmazó HTML elemet
    links = soup.find_all("a", href=True)

    # segít elkerülni a Article download() failed with HTTPSConnectionPool port=443 Read timed out. (read timeout=7) on URL hibákat
    config = Config()
    config.request_timeout = 10

    # végigmegyünk a hírcímeken
    for link in links:

      if link.find(class_="categorized-article-container-title") is not None:
        title = link.find(class_="categorized-article-container-title").text
    # kinyerjük a hírcím szövegét
      href = link["href"]

      if href.startswith(input):
          # létrehozunk egy Article objektumot a linkkel
          article = Article(f'https://origo.hu'+href)
          link = f'https://telex.hu' + article["href"]

          # letöltjük és elemezzük a cikk HTML-jét
          try:
            article.download()
            article.parse()
          # kinyerjük a cikk szövegét
            text = article.text
          # hozzáadjuk a hírt a listához egy szótár formájában
            if (input == '/itthon/'):
              news.append({"title": title, "link": f'https://origo.hu'+href, "text": text})
              #print("news: ", news)
            else:
              neutral_news.append({"title": title, "link": f'https://origo.hu'+href, "text": text})
              #print("neutral news: ", neutral_news)

          except Exception as inst:
            print("Hiba történt egy cikk letöltése vagy elemzése közben")

            print(type(inst))    # a hiba típusa

            print(inst.args)     # arguments stored in .args    ez mit csinál pontosan?



# létrehozunk egy pandas dataframe-et a hírek listájából
df = pd.DataFrame(news)
dfN = pd.DataFrame(neutral_news)

# elmentjük a dataframe-et egy CSV fájlba
df.to_csv("origo_news.csv", index=False)
dfN.to_csv("neutral_news.csv", index=False)


# Telex

In [ ]:
# a kinyerendő cikkek mennyiségének beállítása
# ---------------------------------------------------


# létrehozunk egy üres listát az url-ek tárolására
urls = []
# létrehozunk egy változót kezdőállapotnak
start = 1
# létrehozunk egy változót végállapotnak
end = 3                     # egy napi adag cikk kb 8-9 oldal

# végigmegyünk az oldalakon egy oldalas lépésekkel, amíg el nem érjük a végállapotot
while start < end:
    # átadjuk a kezdőállapotot a page_str-nek
    page_str = f"{start}"
    # összerakjuk az url-t az kezdő oldalszámmal
    url = f"https://telex.hu/archivum?oldal={page_str}"
    # hozzáadjuk az url-t a listához
    urls.append(url)
    # növeljük a kezdőállapotot egy nappal (a ciklus haladásának érdekében)
    start += 1

# kiírjuk az url-ek listáját
print(urls)

['https://telex.hu/archivum?oldal=1']


In [ ]:
# hírek lekérése telexen
# ---------------------------------------------------


#url = 'https://telex.hu/archivum?oldal=1'
# egy üres lista a hírek tárolására
news = []                         # belföldi (politikai) hírek
neutral_news = []                 # egyéb (nem annyira politikai) hírek

# egy lista a választható rovatok tárolásásra
preference = ['/belfold/', '/kult/', '/tudomany/', '/sport/']
#print("preference: ", preference)
input = preference[0]
#print("input: ", input)

# végigmegyünk a weboldalak listáján
for website in urls:
  try:
      # lekérjük a weboldal tartalmát
      response = requests.get(website)
      response.raise_for_status()
  # kiiratjuk a felvetülendő hibát és leállítjuk a programot
  except requests.exceptions.RequestException as e:
      print(e)
      sys.exit(1)

  # létrehozunk egy BeautifulSoup objektumot a HTML elemzéséhez
  soup = BeautifulSoup(response.content, 'html.parser')

  # megkeressük az összes hírcímet tartalmazó HTML elemet
  articles = soup.find_all('a', class_="list__item__title", href=True)

  # segít elkerülni a Article download() failed with HTTPSConnectionPool port=443 Read timed out. (read timeout=7) on URL hibákat
    config = Config()
    config.request_timeout = 10

  #df = pd.DataFrame(columns=['Title', 'Link', 'Text'])

  for article in articles:
      if article['href'].find('/belfold/') == 0:
        title = article.find(class_="hasHighlight").text.strip()
        link = f'https://telex.hu' + article["href"]
        #df = df.append({'title': title, 'link': link}, ignore_index=True)
        article = Article(link)
        # letöltjük és elemezzük a cikk HTML-jét
        try:
          article.download()
          article.parse()
        # kinyerjük a cikk szövegét
          text = article.text
        # hozzáadjuk a hírt a listához egy szótár formájában
        if (input == '/itthon/'):
              news.append({"title": title, "link": f'https://origo.hu'+href, "text": text})
              #print("news: ", news)
            else:
              neutral_news.append({"title": title, "link": f'https://origo.hu'+href, "text": text})
              #print("neutral news: ", neutral_news)

          except Exception as inst:
            print("Hiba történt egy cikk letöltése vagy elemzése közben")

            print(type(inst))    # a hiba típusa

            print(inst.args)     # arguments stored in .args    ez mit csinál pontosan?
        news.append({"title": title, "link": link, "text": text})

      elif article['href'].find('/kult/') == 0 or article['href'].find('/tudomany/') == 0 or article['href'].find('/sport/') == 0:
        title = article.find(class_="hasHighlight").text.strip()
        link = f'https://telex.hu' + article["href"]
        #df = df.append({'title': title, 'link': link}, ignore_index=True)
        article = Article(link)
        # Letöltjük és elemezzük a cikk HTML-jét
        article.download()
        article.parse()
        # Kinyerjük a cikk szövegét
        text = article.text
        # Hozzáadjuk a hírt a listához egy szótár formájában
        neutral_news.append({"title": title, "link": link, "text": text})

df = pd.DataFrame(news)
dfN = pd.DataFrame(neutral_news)
df.to_csv("telex_news.csv", index=False)
dfN.to_csv("neutral_news.csv", index=False)

# Index

# Ripost

# Tárolás csv-ben

In [ ]:
import pandas as pd

origo = pd.read_csv('origo_news.csv')
origo['target'] = origo['target'] = 0
telex = pd.read_csv('telex_news.csv')
telex['target'] = telex['target'] = 1
neutral = pd.read_csv('neutral_news.csv')
neutral['target'] = neutral['target'] = 2

In [ ]:
allnews = pd.concat([origo, telex, neutral])
allnews = allnews[['target','text']]
allnews = allnews.dropna()

In [ ]:
from google.colab import files
import pandas as pd
allnews.to_csv("allnews.csv", index=False)
files.download('allnews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>